In [1]:
from models import FieldMLP, FieldAttention
import importlib
import utils
importlib.reload(utils)
from utils import *

pio.renderers.default = "browser"

In [3]:
circle_sdf = CircleSDF(x0=[0,0], r=0.2)
circle_sdf.plot_field(True)

In [3]:
random = SDF(model=lambda x: torch.asarray(np.random.random((x.shape[0], 1))), xy_lims=(-1, 1, -1, 1))
random.plot_field(True, 5)

In [5]:
square_constraint = Constraints()

@square_constraint.add(5.0, d=0.5, thresh=0.5)
def cont(field, coords, d, thresh=0.5):
    # Calculate whether each point is inside the square
    half_d = d / 2.0
    inside = ((coords[:, 0].abs() <= half_d) & (coords[:, 1].abs() <= half_d)).float().unsqueeze(-1)
    
    p = torch.sigmoid(field)
    target = inside  # 1 inside, 0 outside
    return ((p - (thresh * 0 + target)).abs()).mean()

@square_constraint.add(10.0)
def center(field, coords):
    # Soft weights (could use sigmoid or softmax, depending on field scale)
    p = torch.sigmoid(field)  # shape (N, 1)
    
    # Weighted average position (center of mass)
    weighted_coords = coords * p
    total_weight = p.sum() + 1e-8  # prevent division by zero
    center = weighted_coords.sum(dim=0) / total_weight
    
    # Loss: distance from center to origin
    return center.norm()

@square_constraint.add(5.0, d=0.5, thresh=0.5)
def fill(field, coords, d, thresh=0.5):
    # Calculate whether each point is inside the square
    half_d = d / 2.0
    inside = ((coords[:, 0].abs() <= half_d) & (coords[:, 1].abs() <= half_d)).float().unsqueeze(-1)
    
    p = torch.sigmoid(field)
    target = inside  # 1 inside, 0 outside
    return ((p - target).abs() * target).mean()  # Only penalize points that are inside

In [2]:
circle_constraint = Constraints()

@circle_constraint.add(1.0, r=0.5)
def sdf_loss(field, coords, r):
    x0 = torch.tensor([0.0, 0.0], device=coords.device)
    gt_sdf = torch.linalg.norm(coords - x0, dim=1, keepdim=True) - r
    return (field - gt_sdf).abs().mean()

@circle_constraint.add(0.1)
def eikonal_loss(field, coords):
    grads = torch.autograd.grad(
        outputs=field,
        inputs=coords,
        grad_outputs=torch.ones_like(field),
        create_graph=True,
        retain_graph=True,
        only_inputs=True,
    )[0]
    return ((grads.norm(dim=1) - 1.0) ** 2).mean()

@circle_constraint.add(0.1)
def center_bias(field, coords):
    p = torch.sigmoid(-field)
    weighted_coords = coords * p
    total_weight = p.sum() + 1e-8
    center = weighted_coords.sum(dim=0) / total_weight
    return center.norm()


In [12]:
model = FieldMLP(hidden=128, depth=5).to(device)
sdf = SDF(model=model, xy_lims=(-1, 1, -1, 1), device=device)
wandb_logger = WandbLogger(project="ginn2d", name="run3")
lit = ConstraintTrainer(sdf, constraints=circle_constraint, lr=1e-3, n_points=128, domain=[-1,1], seed=42)
trainer = pl.Trainer(
    max_epochs=100,
    accelerator="auto",
    precision="bf16-mixed" if torch.cuda.is_available() else "32-true",
    logger=wandb_logger,
    gradient_clip_val=1.0,
    log_every_n_steps=10,
)
trainer.fit(lit)

Seed set to 42
Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type     | Params | Mode 
-------------------------------------------
0 | model | FieldMLP | 50.0 K | train
-------------------------------------------
50.0 K    Trainable params
0         Non-trainable params
50.0 K    Total params
0.200     Total estimated model params size (MB)
11        Modules in train mode
0         Modules in eval mode


Epoch 99: 100%|██████████| 10/10 [00:00<00:00, 40.97it/s, v_num=pw1r, loss_step=0.0167, sdf_loss_step=0.0135, eikonal_loss_step=0.00976, center_bias_step=0.023, loss_epoch=0.0232, sdf_loss_epoch=0.0154, eikonal_loss_epoch=0.0143, center_bias_epoch=0.064]   

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|██████████| 10/10 [00:00<00:00, 38.29it/s, v_num=pw1r, loss_step=0.0167, sdf_loss_step=0.0135, eikonal_loss_step=0.00976, center_bias_step=0.023, loss_epoch=0.0232, sdf_loss_epoch=0.0154, eikonal_loss_epoch=0.0143, center_bias_epoch=0.064]


In [13]:
metrics = trainer.logged_metrics
print(metrics)

{'loss_step': tensor(0.0167), 'sdf_loss_step': tensor(0.0135), 'eikonal_loss_step': tensor(0.0098), 'center_bias_step': tensor(0.0230), 'loss_epoch': tensor(0.0232), 'sdf_loss_epoch': tensor(0.0154), 'eikonal_loss_epoch': tensor(0.0143), 'center_bias_epoch': tensor(0.0640)}


In [15]:
sdf.plot_field(
    True, 50, 
    # preprocess = lambda x: torch.sigmoid(x),
)

Attention based model::

In [4]:
model = FieldAttention().to(device)
sdf = SDF(model=model, xy_lims=(-1, 1, -1, 1), device=device)
wandb_logger = WandbLogger(project="ginn2d", name="run_a_1")
lit = ConstraintTrainer(sdf, constraints=circle_constraint, lr=1e-3, n_points=128, domain=[-1,1], seed=42)
trainer = pl.Trainer(
    max_epochs=100,
    accelerator="auto",
    precision="bf16-mixed" if torch.cuda.is_available() else "32-true",
    logger=wandb_logger,
    gradient_clip_val=1.0,
    log_every_n_steps=10,
)
trainer.fit(lit)

Seed set to 42
Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4050 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type           | Params | Mode 
-------------------------------------------------
0 | model | FieldAttention | 30.3 K | train
-------------------------------------------------
30.3 K    Trainable params
0         Non-trainable params
30.3 K    Total params
0.121     Total estimated model params size (MB)
8         Modules in train mode
0         Modules in eval mode
C:\Users\navra\anaconda3\envs\GINN\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:424: PossibleUserWarning:

The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.



Training: |          | 0/? [00:00<?, ?it/s]

wandb: 500 encountered ({"errors":[{"message":"context canceled","path":["upsertBucket"]}],"data":{"upsertBucket":null}}), retrying request


Epoch 99: 100%|██████████| 10/10 [00:00<00:00, 47.99it/s, v_num=kszd, loss_step=0.0142, sdf_loss_step=0.011, eikonal_loss_step=0.00897, center_bias_step=0.0231, loss_epoch=0.0179, sdf_loss_epoch=0.0104, eikonal_loss_epoch=0.0112, center_bias_epoch=0.0639]   

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|██████████| 10/10 [00:00<00:00, 44.74it/s, v_num=kszd, loss_step=0.0142, sdf_loss_step=0.011, eikonal_loss_step=0.00897, center_bias_step=0.0231, loss_epoch=0.0179, sdf_loss_epoch=0.0104, eikonal_loss_epoch=0.0112, center_bias_epoch=0.0639]


In [6]:
sdf.plot_field(
    True, 50, 
    preprocess = lambda x: torch.sigmoid(x),
)